## Initializing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use("seaborn-dark")

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from imblearn.metrics import classification_report_imbalanced

import statsmodels.api as sm
import statsmodels.formula.api as smf

import xgboost as xgb

## Importing Data

In [2]:
df = pd.read_csv('coding_round_data.csv')

### Fixing import issues

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [4]:
for col in ['OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Month']:
    df[col] = df[col].astype('category')

for col in ['Weekend', 'Revenue']:
    df[col] = df[col].astype('int')

## Splitting Data

Using a stratified split as the data is very imbalanced

In [5]:
df_train, df_test = train_test_split(df, stratify=df['Revenue'], test_size=0.20, random_state=42)

## Modelling

There are 2 main approaches that I will be taking for modelling. Their results will be compared on the df_test set to decide a final approach

### Approach 1 - Logistic Regression

In [6]:
formula = "Revenue ~ " + " + ".join(df_train.columns.drop('Revenue'))

LR_model = smf.logit(formula=formula, data=df_train).fit_regularized(method='l1')
LR_model.summary()

/home/kavish/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/kavish/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1863: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.2831479736005282
            Iterations: 444
            Function evaluations: 465
            Gradient evaluations: 444


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Revenue   No. Observations:                 9864
Model:                          Logit   Df Residuals:                     9795
Method:                           MLE   Df Model:                           68
Date:                Sun, 03 Apr 2022   Pseudo R-squ.:                  0.3427
Time:                        17:06:08   Log-Likelihood:                -2793.0
converged:                       True   LL-Null:                       -4249.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -1.9108        nan        nan        nan         nan         nan
Month[T.Dec]                        -0.7004      0.175     -3.996      0.000      -1.044      -0.357
Month[T.Feb]                        -2.0139      0.620     -3.249      0.001      -3.229      -0.799
Month[T.Jul]                        -0.1133      0.447     -0.254      0.800      -0.989       0.762
Month[T.June]                       -0.3465      0.491     -0.706      0.480      -1.308       0.615
Month[T.Mar]                        -0.6664      0.218     -3.058      0.002      -1.093      -0.239
Month[T.May]                        -0.5919      0.250     -2.369      0.018      -1.082      -0.102
Month[T.Nov]                         0.3265      0.129      2.539      0.011       0.074       0.578
Month[T.Oct]                        -0.0968      0.160     -0.607      0.544      -0.410       0.216
Month[T.Sep]                         0.0388      0.117      0.332      0.740      -0.190       0.268
OperatingSystems[T.2]                0.1653      0.196      0.842      0.400      -0.219       0.550
OperatingSystems[T.3]               -0.0753      0.201     -0.374      0.708      -0.470       0.320
OperatingSystems[T.4]               -0.0583      0.202     -0.289      0.773      -0.454       0.337
OperatingSystems[T.5]                0.0629   4.42e+16   1.42e-18      1.000   -8.67e+16    8.67e+16
OperatingSystems[T.6]               -0.9505      0.979     -0.971      0.332      -2.869       0.968
OperatingSystems[T.7]                1.6387      1.237      1.324      0.185      -0.787       4.064
OperatingSystems[T.8]                0.1174      0.733      0.160      0.873      -1.319       1.554
Browser[T.2]                        -0.1261      0.187     -0.675      0.499      -0.492       0.240
Browser[T.3]                        -0.8553      0.597     -1.432      0.152      -2.026       0.315
Browser[T.4]                        -0.0639      0.237     -0.270      0.788      -0.528       0.401
Browser[T.5]                         0.0550      0.256      0.215      0.830      -0.446       0.556
Browser[T.6]                        -0.3320      0.370     -0.897      0.370      -1.057       0.393
Browser[T.7]                        -1.2444      0.806     -1.543      0.123      -2.825       0.336
Browser[T.8]                        -0.4327      0.444     -0.974      0.330      -1.303       0.438
Browser[T.9]                        -0.4633      7.495     -0.062      0.951     -15.153      14.226
Browser[T.10]                        0.2741      0.318      0.861      0.389      -0.350       0.898
Browser[T.11]                        0.0629   4.42e+16   1.42e-18      1.000   -8.67e+16    8.67e+16
Browser[T.12]                        2.4418      0.881      2.770      0.006       0.714       4.169
Browser[T.13]                        0.5204      1.122      0.464      0.643      -1.678       2.719
Region[T.

As the classes are not balanced, the intercept/threshold needs to be scaled accordingly.

In [7]:
weights = df_train.Revenue.value_counts()

threshold = weights[1]/weights[0]

LR_test_preds_raw = LR_model.predict(exog=df_test.drop(columns=['Revenue']))
LR_test_preds = (LR_test_preds_raw > threshold).astype('int')

print(classification_report_imbalanced(df_test['Revenue'], LR_test_preds))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.87      0.71      0.91      0.79      0.63      2084
          1       0.50      0.71      0.87      0.59      0.79      0.61       382

avg / total       0.88      0.85      0.74      0.86      0.79      0.63      2466

